### benchmark：neural network

In [1]:
import gc
import numpy as np 
import pandas as pd
import warnings
import os


warnings.filterwarnings('ignore')
# %matplotlib inline


print(os.curdir)
print(os.listdir("input"))


.
['train_data_c.csv', 'train_labels.csv', 'test_data_100.csv', '.DS_Store', 'train_data_100.csv', 'train_data.csv', 'memo.txt', 'sample_submission.csv']


In [2]:
%%time
h5File = "temp_data/h5/selectkbest_data_with_last.h5";
key_train = 'bestkselect_01/train';
key_valid = 'bestkselect_01/valid';
key_test = 'bestkselect_01/test';

train = pd.read_hdf(h5File, key_train);
valid = pd.read_hdf(h5File, key_valid);
test = pd.read_hdf(h5File, key_test);


CPU times: user 44 s, sys: 1min 44s, total: 2min 28s
Wall time: 4min 18s


72

In [3]:
%%time
h5File = "temp_data/h5/selectkbest_data_with_last_scaled.h5";
key_train = 'bestkselect_01_scaled/train';
key_test = 'bestkselect_01_scaled/test';

import h5py
with h5py.File(h5File, 'r') as f:
    train_scaled = f[key_train][()]
    test_scaled = f[key_test][()]


CPU times: user 63.7 ms, sys: 19 s, total: 19.1 s
Wall time: 25.8 s


In [4]:
# X_train = train[:10000]
# y_train = valid[:10000]
# X_test = test[:10000]

In [5]:
X_train = train
y_train = valid
X_test = test

In [6]:
X_train.shape

(458913, 1009)

In [7]:
y_train.shape

(458913,)

In [8]:
X_test.shape

(924621, 1009)

In [9]:
gc.collect()

0

In [10]:
%%time
from sklearn.model_selection import train_test_split

# ※スケールデータを使っている
X_tr_scaled, X_val_scaled, y_tr, y_val = train_test_split(train_scaled, y_train, test_size=0.3, random_state=0, stratify=y_train)
print('=== X_tr:', X_tr_scaled[:10])
print('=== X_val:',X_val_scaled[:10])
print('=== y_tr:',y_tr.head())
print('=== y_val:',y_val.head())


=== X_tr: [[-3.08526483 -3.29899639 -0.44749445 ...  0.          0.
  -0.98045311]
 [ 0.36265354  0.31900395  0.5110827  ...  0.          0.
  -0.2863815 ]
 [ 0.36265354  0.65585226  1.18120728 ...  0.          0.
   1.21744031]
 ...
 [ 0.36265354  0.28157636  1.12406235 ...  0.          0.
  -0.6334173 ]
 [ 0.36265354  0.20672118 -0.86397804 ...  0.          0.
  -1.55884611]
 [ 0.36265354  0.39385913  0.13122557 ...  0.          0.
   0.0606543 ]]
=== X_val: [[ 0.36265354  0.33147981  1.47354617 ...  0.          0.
  -1.44316751]
 [-0.0204485  -0.01784436  1.25033321 ...  0.          0.
   1.10176171]
 [ 0.36265354  0.31900395  0.20537951 ...  0.          0.
   0.2920115 ]
 ...
 [ 0.36265354  0.41881086  0.7836207  ...  0.          0.
   1.10176171]
 [ 0.36265354  0.55604535  0.1607167  ...  0.          0.
   0.52336871]
 [ 0.36265354  0.34395568 -1.70957108 ...  0.          0.
   1.56447612]]
=== y_tr: customer_ID
6c0333b9b0fac08bbdafa1bab52c58a83885b5f7d08c7be568fadc1000520066    1

In [11]:
gc.collect()

0

In [12]:
%%time

from keras.layers import Dense, Dropout
from keras.models import Sequential
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler


CPU times: user 4.47 s, sys: 1.26 s, total: 5.72 s
Wall time: 10.8 s


In [13]:
# import tensorflow as tf
# tf.config.run_functions_eagerly(False)
# tf.config.experimental_run_functions_eagerly(False)
# tf.executing_eagerly()

In [14]:
%%time


# ニューラルネットモデルの構築
print('# ニューラルネットモデルの構築')
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'], run_eagerly=True)

# 学習の実行
print('# 学習の実行')
# バリデーションモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
batch_size = 128
epochs = 10
history = model.fit(X_tr_scaled, y_tr, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_val_scaled, y_val))

#バリデーションデータでのスコアの確認
print('# バリデーションデータでのスコアの確認')
va_pred = model.predict(X_val_scaled)
score = log_loss(y_val, va_pred, eps=1e-7)
print(f'logloss: {score:.4f}')




# ニューラルネットモデルの構築
# 学習の実行


2022-08-21 10:27:35.686304: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
2510/2510 [==============================] - 79s 31ms/step - loss: 0.2346 - accuracy: 0.8964 - val_loss: 0.2265 - val_accuracy: 0.8995
Epoch 2/10
2510/2510 [==============================] - 61s 24ms/step - loss: 0.2247 - accuracy: 0.9017 - val_loss: 0.2268 - val_accuracy: 0.8995
Epoch 3/10
2510/2510 [==============================] - 61s 24ms/step - loss: 0.2215 - accuracy: 0.9033 - val_loss: 0.2255 - val_accuracy: 0.9011
Epoch 4/10
2510/2510 [==============================] - 58s 23ms/step - loss: 0.2183 - accuracy: 0.9047 - val_loss: 0.2256 - val_accuracy: 0.9015
Epoch 5/10
2510/2510 [==============================] - 62s 25ms/step - loss: 0.2143 - accuracy: 0.9065 - val_loss: 0.2264 - val_accuracy: 0.9009
Epoch 6/10
2510/2510 [==============================] - 55s 22ms/step - loss: 0.2111 - accuracy: 0.9079 - val_loss: 0.2265 - val_accuracy: 0.9009
Epoch 7/10
2510/2510 [==============================] - 63s 25ms/step - loss: 0.2063 - accuracy: 0.9097 - val_loss: 0.2311 -

In [15]:
gc.collect()

2145

In [16]:
print(f'logloss: {score:.4f}')


logloss: 0.2400


In [17]:
%%time
#予測
# ※スケールされたデータを使っている
%time y_pred = model.predict(test_scaled)


28895/28895 [==============================] - 102s 4ms/step
CPU times: user 2min 20s, sys: 1min 3s, total: 3min 23s
Wall time: 2min 42s
CPU times: user 2min 20s, sys: 1min 3s, total: 3min 23s
Wall time: 2min 42s


In [18]:
y_pred

array([[5.1261869e-04],
       [7.5567455e-04],
       [8.5266575e-02],
       ...,
       [6.5795386e-01],
       [4.1049847e-01],
       [1.2981190e-02]], dtype=float32)

In [19]:
sub = pd.read_csv('input/sample_submission.csv')
sub

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0


In [20]:
X_test

,count,S_2_days,P_2_min,P_2_max,P_2_avg,D_39_min,D_39_max,D_39_avg,B_1_min,B_1_max,...,D_144_last,D_144_ldiff,D_145_last,D_145_ldiff,S_2_min_y,S_2_min_m,S_2_min_d,S_2_max_y,S_2_max_m,S_2_max_d
customer_ID,,,,,,,,,,,,,,,,,,,,,
00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,9,235,0.568930,0.631315,0.601387,0.001912,0.238794,0.069758,0.007959,0.021672,...,0.003690,-0.002040,0.003219,-0.002052,2019,2,19,2019,10,12
00001bf2e77ff879fab36aa4fac689b9ba411dae63ae397d4263dafa1daedef5,13,358,0.794469,0.913501,0.862166,0.001548,0.505795,0.154252,0.014187,0.276014,...,0.000247,-0.005082,0.007780,0.002313,2018,4,22,2019,4,15
0000210045da4f81e5f122c6bde5c2a617d03eef67f82c5e400fc98e7bd43ce8,13,378,0.673112,0.835114,0.748955,0.000482,0.679547,0.181814,0.001483,0.039697,...,0.457819,0.070535,0.092041,-0.004207,2018,10,3,2019,10,16
00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976cf6e56734528702d694,13,382,0.428457,0.514222,0.474728,0.212508,0.683640,0.470512,0.149511,0.309129,...,0.500924,0.061332,0.183020,-0.003673,2018,4,5,2019,4,22
00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9a4693dd914fca22557,13,370,0.254478,0.425764,0.324100,0.035067,0.768016,0.353050,0.357828,0.563603,...,0.001558,-0.002867,0.000525,-0.004507,2018,10,17,2019,10,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c5d60460dba6dedc41e,13,365,0.597667,0.646915,0.631613,0.000079,0.009711,0.003932,0.001874,0.011309,...,0.003278,-0.001440,0.005294,0.000760,2018,4,14,2019,4,14
ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3a4f0ca3de613b0b2ad,13,371,0.471303,0.634783,0.547049,0.000903,0.417259,0.056910,0.015243,0.084163,...,0.005432,0.000243,0.009981,0.004037,2018,10,13,2019,10,19
ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475cb095d2443a68030f1,13,357,0.195671,0.316648,0.239070,0.001037,1.119398,0.232943,0.013811,0.051597,...,0.004849,0.000825,0.002120,-0.001577,2018,4,14,2019,4,6


In [21]:
# create submission
X_tiny = X_test.iloc[:,[0]]
X_tiny = pd.merge(X_tiny, sub, left_index=True, right_on='customer_ID')
X_tiny['prediction'] = y_pred.round(2)
X_tiny = X_tiny.drop(columns=X_tiny.columns[[0]])
sub = X_tiny
sub

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.00
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.00
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.09
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.18
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.98
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.01
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.88
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.66
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.41


In [22]:
sub.to_csv('output/submission/sub_benchmark_plus_nn.csv', index=False)
